## Image Interpolation

In [1]:
import cv2
import os
import numpy as np

# Paths
input_folder = "stage4"
output_folder = "stage4-interpolated"
os.makedirs(output_folder, exist_ok=True)

def interpolate_frames(frame1, frame2, num_interpolations=3):
    """
    Interpolates between two frames using linear interpolation.
    
    Args:
        frame1 (numpy.ndarray): First frame.
        frame2 (numpy.ndarray): Second frame.
        num_interpolations (int): Number of interpolated frames to generate.
    
    Returns:
        list: List of interpolated frames.
    """
    interpolated_frames = []
    for i in range(1, num_interpolations + 1):
        alpha = i / (num_interpolations + 1)  # Interpolation ratio
        interpolated_frame = cv2.addWeighted(frame1, 1 - alpha, frame2, alpha, 0)
        interpolated_frames.append(interpolated_frame)
    return interpolated_frames

def process_video(video_path, output_path, num_interpolations=3):
    """
    Processes a video to add interpolated frames and saves the result.
    
    Args:
        video_path (str): Path to the input video.
        output_path (str): Path to save the interpolated video.
        num_interpolations (int): Number of interpolated frames between each pair of original frames.
    """
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"Failed to open video: {video_path}")
        return
    
    # Get video properties
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Codec for .mp4 files
    
    # New FPS after interpolation
    new_fps = fps * (num_interpolations + 1)
    out = cv2.VideoWriter(output_path, fourcc, new_fps, (width, height))
    
    ret, prev_frame = cap.read()
    while ret:
        ret, next_frame = cap.read()
        if not ret:
            break

        # Write the original frame
        out.write(prev_frame)

        # Generate and write interpolated frames
        interpolated_frames = interpolate_frames(prev_frame, next_frame, num_interpolations)
        for frame in interpolated_frames:
            out.write(frame)
        
        # Update the previous frame
        prev_frame = next_frame

    # Release resources
    cap.release()
    out.release()
    print(f"Interpolated video saved to: {output_path}")

# Process all videos in the input folder
num_interpolations = 3  # Number of interpolated frames between each pair

for video_file in os.listdir(input_folder):
    if video_file.endswith(".mp4"):
        input_path = os.path.join(input_folder, video_file)
        output_path = os.path.join(output_folder, video_file)
        process_video(input_path, output_path, num_interpolations=num_interpolations)

print("Interpolation complete for all videos.")


Interpolated video saved to: stage4-interpolated\user1.mp4
Interpolated video saved to: stage4-interpolated\user10.mp4
Interpolated video saved to: stage4-interpolated\user12.mp4
Interpolated video saved to: stage4-interpolated\user13.mp4
Interpolated video saved to: stage4-interpolated\user19.mp4
Interpolated video saved to: stage4-interpolated\user2.mp4
Interpolated video saved to: stage4-interpolated\user20.mp4
Interpolated video saved to: stage4-interpolated\user22.mp4
Interpolated video saved to: stage4-interpolated\user23.mp4
Interpolated video saved to: stage4-interpolated\user3.mp4
Interpolated video saved to: stage4-interpolated\user5.mp4
Interpolated video saved to: stage4-interpolated\user8.mp4
Interpolation complete for all videos.


## Frames count

In [2]:
import cv2
import os

# Path to the folder containing the videos
stage_path = "stage4-interpolated"

# List to store the number of frames for each video
frame_counts = []

# Iterate through the videos in the folder
for file in os.listdir(stage_path):
    if file.endswith(".mp4"):
        video_file_path = os.path.join(stage_path, file)
        
        # Open the video file
        cap = cv2.VideoCapture(video_file_path)
        
        if not cap.isOpened():
            print(f"Error opening video file: {file}")
            continue

        # Get the total number of frames
        total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        frame_counts.append(total_frames)

        cap.release()

# Calculate average, minimum, and maximum frame counts
if frame_counts:
    avg_frames = sum(frame_counts) / len(frame_counts)
    min_frames = min(frame_counts)
    max_frames = max(frame_counts)

    print(f"Average frames per video: {avg_frames:.2f}")
    print(f"Minimum frames in a video: {min_frames}")
    print(f"Maximum frames in a video: {max_frames}")
else:
    print("No valid videos found in the specified folder.")


Average frames per video: 219.33
Minimum frames in a video: 120
Maximum frames in a video: 356


## Standardize videos

In [3]:
import cv2
import os

# Path to input and output directories
input_dir = "stage4-interpolated"
output_dir = "stage4-200frames"
os.makedirs(output_dir, exist_ok=True)

target_frames = 200  # Target number of frames per video

def truncate_or_pad_video(video_path, output_path, target_frames):
    """
    Truncate or pad a video to ensure it has exactly `target_frames` frames.

    Args:
        video_path (str): Path to the input video.
        output_path (str): Path to save the processed video.
        target_frames (int): Desired number of frames.
    """
    cap = cv2.VideoCapture(video_path)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))

    # Create VideoWriter for the output video
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))

    frames = []
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        frames.append(frame)
    cap.release()

    if len(frames) < target_frames:
        # Pad by duplicating the last frame until reaching the target
        while len(frames) < target_frames:
            frames.append(frames[-1])  # Duplicate the last frame
    elif len(frames) > target_frames:
        # Truncate the video to the target number of frames
        frames = frames[:target_frames]

    # Write the processed frames to the output video
    for frame in frames:
        out.write(frame)
    out.release()
    print(f"Processed: {video_path} -> {output_path} with {target_frames} frames.")

# Process all videos in the input directory
for file in os.listdir(input_dir):
    if file.endswith(".mp4"):
        input_path = os.path.join(input_dir, file)
        output_path = os.path.join(output_dir, file)
        truncate_or_pad_video(input_path, output_path, target_frames)

print("All videos resized to exactly 200 frames using truncate or pad method.")


Processed: stage4-interpolated\user1.mp4 -> stage4-200frames\user1.mp4 with 200 frames.
Processed: stage4-interpolated\user10.mp4 -> stage4-200frames\user10.mp4 with 200 frames.
Processed: stage4-interpolated\user12.mp4 -> stage4-200frames\user12.mp4 with 200 frames.
Processed: stage4-interpolated\user13.mp4 -> stage4-200frames\user13.mp4 with 200 frames.
Processed: stage4-interpolated\user19.mp4 -> stage4-200frames\user19.mp4 with 200 frames.
Processed: stage4-interpolated\user2.mp4 -> stage4-200frames\user2.mp4 with 200 frames.
Processed: stage4-interpolated\user20.mp4 -> stage4-200frames\user20.mp4 with 200 frames.
Processed: stage4-interpolated\user22.mp4 -> stage4-200frames\user22.mp4 with 200 frames.
Processed: stage4-interpolated\user23.mp4 -> stage4-200frames\user23.mp4 with 200 frames.
Processed: stage4-interpolated\user3.mp4 -> stage4-200frames\user3.mp4 with 200 frames.
Processed: stage4-interpolated\user5.mp4 -> stage4-200frames\user5.mp4 with 200 frames.
Processed: stage4-

## Augmentation

In [4]:
import cv2
import os
import pandas as pd
import numpy as np

# Paths
input_folder = "stage4-200frames"
output_folder = "stage4-dataset"
os.makedirs(output_folder, exist_ok=True)

# Load labels
labels_df = pd.read_csv("stage4.csv")  # Ensure this CSV contains 'video' and 'label' columns

# Augmentation Functions
def augment_video(video_path, output_path, augmentation_type, is_odd):
    """
    Augment the video with the specified augmentation type and save it.
    
    Args:
        video_path (str): Path to the input video.
        output_path (str): Path to save the augmented video.
        augmentation_type (str): Type of augmentation ('rotate', 'brightness', 'noise', etc.).
        is_odd (bool): Whether the video index is odd or even.
    """
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"Error: Unable to open video file {video_path}")
        return

    # Get video properties
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Codec for .mp4 files

    # Create VideoWriter object
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Apply augmentation
        if augmentation_type == "mirrored":
            frame = cv2.flip(frame, 1)  # Horizontal flip
        elif augmentation_type == "rotate":
            angle = 3 if is_odd else -3  # Adjusted rotation angles
            M = cv2.getRotationMatrix2D((width // 2, height // 2), angle, 1)
            frame = cv2.warpAffine(frame, M, (width, height))
        elif augmentation_type == "brightness":
            alpha = 1.05 if is_odd else 0.95  # Adjusted brightness
            frame = cv2.convertScaleAbs(frame, alpha=alpha, beta=0)
        elif augmentation_type == "noise":
            noise = np.random.normal(0, 15, frame.shape).astype(np.uint8)  # Add small random noise
            frame = cv2.add(frame, noise)

        # Write the augmented frame
        out.write(frame)

    cap.release()
    out.release()

# Augment videos and create a new CSV file
augmented_data = []
for idx, row in labels_df.iterrows():
    video_path = row['video']
    label = row['label']
    base_name = os.path.splitext(os.path.basename(video_path))[0]

    # Original video path
    input_video_path = os.path.join(input_folder, os.path.basename(video_path))
    is_odd = (idx % 2 == 1)

    # Add original video to the dataset
    original_video_path = os.path.join(output_folder, f"{base_name}_original.mp4")
    if not os.path.exists(original_video_path):
        cap = cv2.VideoCapture(input_video_path)
        if cap.isOpened():
            width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
            height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
            fps = int(cap.get(cv2.CAP_PROP_FPS))
            fourcc = cv2.VideoWriter_fourcc(*'mp4v')
            out = cv2.VideoWriter(original_video_path, fourcc, fps, (width, height))
            while cap.isOpened():
                ret, frame = cap.read()
                if not ret:
                    break
                out.write(frame)
            cap.release()
            out.release()
    augmented_data.append({
        "video": original_video_path,
        "label": label
    })

    # Apply augmentations
    for aug_type in ["mirrored", "rotate", "brightness", "noise"]:
        output_video_name = f"{base_name}_{aug_type}.mp4"
        output_video_path = os.path.join(output_folder, output_video_name)
        augment_video(input_video_path, output_video_path, aug_type, is_odd)

        # Append to dataset
        augmented_data.append({
            "video": output_video_path,
            "label": label
        })

# Create a new DataFrame for the augmented dataset
augmented_df = pd.DataFrame(augmented_data)

# Save the new dataset labels to a CSV file
augmented_df.to_csv("stage4-dataset.csv", index=False)
print("Augmented videos saved to stage4-dataset and labels saved to stage4-dataset.csv.")


Augmented videos saved to stage4-dataset and labels saved to stage4-dataset.csv.


## Keypoints extraction

In [5]:
import os
import cv2
import mediapipe as mp
import numpy as np
import json

# Paths
input_videos_path = "stage4-dataset"  # Path to input videos
output_json_path = "stage4-dataset-json"  # Path to save JSON files
os.makedirs(output_json_path, exist_ok=True)

# Initialize MediaPipe Pose model
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=False, min_detection_confidence=0.5, min_tracking_confidence=0.5)

def extract_keypoints(video_path, output_json_path, frame_skip=2):
    """
    Extracts x and y coordinates of key landmarks and saves them in a JSON file, skipping frames.
    
    Args:
        video_path (str): Path to the input video.
        output_json_path (str): Path to save extracted keypoints.
        frame_skip (int): Process 1 out of 'frame_skip' frames.
    """
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"Error: Unable to open video file {video_path}")
        return

    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    frame_count = 0

    keypoints_list = []

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        frame_count += 1

        # Skip every second frame
        if frame_count % frame_skip != 0:
            continue

        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = pose.process(frame_rgb)

        if results.pose_landmarks:
            landmarks = results.pose_landmarks.landmark

            # Extract x, y values for selected keypoints (normalized values 0-1)
            nose_x, nose_y = landmarks[mp_pose.PoseLandmark.NOSE].x, landmarks[mp_pose.PoseLandmark.NOSE].y
            left_shoulder_x, left_shoulder_y = landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER].x, landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER].y
            right_shoulder_x, right_shoulder_y = landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER].x, landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER].y
            left_hip_x, left_hip_y = landmarks[mp_pose.PoseLandmark.LEFT_HIP].x, landmarks[mp_pose.PoseLandmark.LEFT_HIP].y
            right_hip_x, right_hip_y = landmarks[mp_pose.PoseLandmark.RIGHT_HIP].x, landmarks[mp_pose.PoseLandmark.RIGHT_HIP].y

            # Store extracted keypoints in normalized values (0 to 1)
            keypoints_list.append({
                "frame": int(cap.get(cv2.CAP_PROP_POS_FRAMES)),
                "nose": [nose_x, nose_y],
                "left_shoulder": [left_shoulder_x, left_shoulder_y],
                "right_shoulder": [right_shoulder_x, right_shoulder_y],
                "left_hip": [left_hip_x, left_hip_y],
                "right_hip": [right_hip_x, right_hip_y],
            })

    cap.release()

    # Save extracted keypoints to a JSON file
    with open(output_json_path, "w") as f:
        json.dump(keypoints_list, f, indent=4)

    print(f"Keypoints extracted and saved to: {output_json_path}")

# Process all videos in the input folder
for video_file in os.listdir(input_videos_path):
    if video_file.endswith(".mp4"):
        video_path = os.path.join(input_videos_path, video_file)
        json_filename = os.path.splitext(video_file)[0] + "_keypoints.json"
        json_output_path = os.path.join(output_json_path, json_filename)

        extract_keypoints(video_path, json_output_path, frame_skip=2)

print("All videos processed. Keypoints saved to JSON.")


Keypoints extracted and saved to: stage4-dataset-json\user10_brightness_keypoints.json
Keypoints extracted and saved to: stage4-dataset-json\user10_mirrored_keypoints.json
Keypoints extracted and saved to: stage4-dataset-json\user10_noise_keypoints.json
Keypoints extracted and saved to: stage4-dataset-json\user10_original_keypoints.json
Keypoints extracted and saved to: stage4-dataset-json\user10_rotate_keypoints.json
Keypoints extracted and saved to: stage4-dataset-json\user12_brightness_keypoints.json
Keypoints extracted and saved to: stage4-dataset-json\user12_mirrored_keypoints.json
Keypoints extracted and saved to: stage4-dataset-json\user12_noise_keypoints.json
Keypoints extracted and saved to: stage4-dataset-json\user12_original_keypoints.json
Keypoints extracted and saved to: stage4-dataset-json\user12_rotate_keypoints.json
Keypoints extracted and saved to: stage4-dataset-json\user13_brightness_keypoints.json
Keypoints extracted and saved to: stage4-dataset-json\user13_mirrored

In [4]:
import os
import json
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
import pandas as pd

# Paths
json_folder = "stage4-dataset-json"  # Folder containing JSON files with extracted features
labels_file = "stage4-dataset.csv"  # CSV file with video labels (e.g., user1.mp4: 0, user2.mp4: 1)

# Load labels (assumes a CSV file with 'video' and 'label' columns)
labels_df = pd.read_csv(labels_file)
label_mapping = dict(zip(labels_df['video'].str.replace(".mp4", ""), labels_df['label']))

# Function to calculate Euclidean distance between two points
def calculate_distance(p1, p2):
    return np.sqrt((p2[0] - p1[0]) ** 2 + (p2[1] - p1[1]) ** 2)

# Function to calculate the angle formed by three points (b is the vertex)
def calculate_angle(a, b, c):
    a, b, c = np.array(a), np.array(b), np.array(c)
    ba = a - b
    bc = c - b
    cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
    angle = np.arccos(np.clip(cosine_angle, -1.0, 1.0))
    return np.degrees(angle)

# Dataset preparation
X = []  # Features
y = []  # Labels
max_sequence_length = 200  # Fixed sequence length for LSTM or Transformer

for json_file in sorted(os.listdir(json_folder)):
    user_id = os.path.splitext(json_file)[0]  # Extract user ID from file name without .json
    if user_id not in label_mapping:
        print(f"Label not found for {user_id}. Skipping...")
        continue

    label = label_mapping[user_id]  # Get label for the user
    json_path = os.path.join(json_folder, json_file)

    # Load JSON data
    with open(json_path, "r") as file:
        data = json.load(file)

    # Extract meaningful features
    user_features = []
    for frame in data:
        try:
            # Extract x and y values for selected keypoints
            nose = frame["nose"]
            left_shoulder = frame["left_shoulder"]
            right_shoulder = frame["right_shoulder"]
            left_hip = frame["left_hip"]
            right_hip = frame["right_hip"]

            # Compute meaningful features
            shoulder_width = calculate_distance(left_shoulder, right_shoulder)
            hip_width = calculate_distance(left_hip, right_hip)
            torso_length = calculate_distance(left_shoulder, left_hip)
            shoulder_hip_angle = calculate_angle(left_shoulder, nose, left_hip)
            hip_movement = right_hip[0] - left_hip[0]  # Track x movement

            # Combine features into a single vector
            features = [
                shoulder_width,  # Shoulder width
                hip_width,       # Hip width
                torso_length,    # Torso length
                shoulder_hip_angle,  # Angle from shoulder to hip
                hip_movement     # X-axis movement tracking
            ]
            user_features.append(features)
        except KeyError as e:
            print(f"Missing key {e} in {json_file}. Skipping frame...")
            continue

    # Append features and label if data exists
    if user_features:
        X.append(user_features)
        y.append(label)

# Pad sequences to ensure consistent lengths
X = pad_sequences(X, maxlen=max_sequence_length, dtype='float32', padding='post', truncating='post')

# Convert to numpy arrays
X = np.array(X)
y = np.array(y)

# Split into train and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, random_state=0, stratify=y)

# Model Constants
sequence_length = X_train.shape[1]  # Fixed length of sequences
num_features = X_train.shape[2]  # Number of features per frame
num_classes = len(np.unique(y_train))  # Number of unique labels

print(f"Dataset prepared:")
print(f"Training set shape: {X_train.shape}, Validation set shape: {X_val.shape}")
print(f"Number of features per frame: {num_features}")
print(f"Number of classes: {num_classes}")


Dataset prepared:
Training set shape: (40, 200, 5), Validation set shape: (18, 200, 5)
Number of features per frame: 5
Number of classes: 2


In [8]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.compat.v1.keras.layers import LSTM
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

# Calculate class weights to handle data imbalance
class_weights = compute_class_weight(
    class_weight="balanced", 
    classes=np.unique(y_train), 
    y=y_train
)
class_weights_dict = dict(enumerate(class_weights))
print(f"Class weights: {class_weights_dict}")

# Build the LSTM model
model = Sequential([
    LSTM(128, activation='tanh', return_sequences=True, input_shape=(sequence_length, num_features)),
    Dropout(0.3),
    LSTM(64, activation='tanh', return_sequences=False),  # Add L2 regularization
    Dropout(0.3),
    Dense(64, activation='relu'),  # Add L2 regularization to dense layer
    Dropout(0.2),
    Dense(1, activation='sigmoid')  # Output: Value between 0 and 1
])

model.compile(
    optimizer=Adam(learning_rate=0.01), 
    loss='binary_crossentropy',  # Using binary crossentropy for binary classification
    metrics=['accuracy']  # Train on accuracy metric
)


# Early stopping to prevent overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Train the model with class weights
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=60,  # Adjust based on dataset size
    batch_size=16,  # Adjust based on memory constraints
    class_weight=class_weights_dict,  # Use class weights to address imbalance
   #  callbacks=[early_stopping],  # Early stopping to prevent overfitting
    verbose=1
)

# Save the model
model.save("stage4-final.keras")
print("Model training complete and saved as 'stage3-spear.keras'")


Class weights: {0: 1.1764705882352942, 1: 0.8695652173913043}
Epoch 1/60
3/3 [==============================] - 10s 3s/step - loss: 0.7023 - accuracy: 0.5250 - val_loss: 0.6911 - val_accuracy: 0.5556
Epoch 2/60
3/3 [==============================] - 8s 3s/step - loss: 0.7232 - accuracy: 0.3750 - val_loss: 0.6933 - val_accuracy: 0.4444
Epoch 3/60
3/3 [==============================] - 8s 3s/step - loss: 0.6945 - accuracy: 0.5000 - val_loss: 0.6939 - val_accuracy: 0.4444
Epoch 4/60
3/3 [==============================] - 8s 3s/step - loss: 0.6918 - accuracy: 0.5750 - val_loss: 0.6885 - val_accuracy: 0.5556
Epoch 5/60
3/3 [==============================] - 8s 3s/step - loss: 0.7127 - accuracy: 0.5500 - val_loss: 0.6900 - val_accuracy: 0.5556
Epoch 6/60
3/3 [==============================] - 7s 2s/step - loss: 0.6960 - accuracy: 0.5500 - val_loss: 0.6914 - val_accuracy: 0.5556
Epoch 7/60
3/3 [==============================] - 7s 2s/step - loss: 0.6921 - accuracy: 0.5000 - val_loss: 0.6957 -